In [1]:
from connect import get_odps
from get_data import get_data, delete_output
from get_df import get_dataframe
from read_data import read_data
from transform import transform
from get_dataset import get_dataset
from context_vector import context_vec, embedding_vec

import numpy as np
import pandas as pd

import sklearn.cluster as sc
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from cluster.KMeans import KMeans, EuclidianDistance, CosineDistance
import numpy as np

import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from MLP import MLP
from TextCNN import TextCNN

o = get_odps()

odps.Schema {
  login                     string        
  created_at                date          
  database_id               int64         
  location                  string        
  company                   string        
  bio                       string        
  is_employee               boolean       
  email                     string        
  infoname                  string        
  followers                 string        
  following                 string        
  time                      date          
  name                      string        
  lastupdatedat             date          
  nextupdateat              date          
}

odps.Schema {
  id                                                                          string                  
  type                                                                        string                  
  action                                                                      string                  
  actor_id     

In [2]:
df = get_dataframe(o)


                      0                    1                             2  \
0  (repo_id, 159513310)  (actor_id, 9887585)     (actor_login, snellingio)   
1  (repo_id, 167922316)  (actor_id, 9887585)     (actor_login, snellingio)   
2  (repo_id, 170486632)  (actor_id, 9887585)     (actor_login, snellingio)   
3  (repo_id, 105722352)  (actor_id, 9887716)       (actor_login, vpperego)   
4  (repo_id, 158469400)  (actor_id, 9887743)  (actor_login, ElliottMiller)   

                          3                  4                 5  
0        (type, WatchEvent)  (action, started)  (pull_merged, 0)  
1        (type, WatchEvent)  (action, started)  (pull_merged, 0)  
2        (type, WatchEvent)  (action, started)  (pull_merged, 0)  
3        (type, WatchEvent)  (action, started)  (pull_merged, 0)  
4  (type, PullRequestEvent)   (action, opened)  (pull_merged, 0)  


In [3]:
arr = np.array(df)

for i in range(100000):
    for j in range(0,5):
        if(len(arr[i][j])) == 2:
            arr[i][j] = arr[i][j][1]
print(arr)
df = pd.DataFrame(arr)
df.columns = ['repo_id', 'actor_id', 'actor_login', 'type', 'action', 'pull_merged']
print(df.head())
df.to_csv('Action.csv')

[[159513310 9887585 'snellingio' 'WatchEvent' 'started'
  ('pull_merged', 0)]
 [167922316 9887585 'snellingio' 'WatchEvent' 'started'
  ('pull_merged', 0)]
 [170486632 9887585 'snellingio' 'WatchEvent' 'started'
  ('pull_merged', 0)]
 ...
 [66922941 1029811 'ToBeFree' 'WatchEvent' 'started' ('pull_merged', 0)]
 [111535895 1029811 'ToBeFree' 'WatchEvent' 'started' ('pull_merged', 0)]
 [166080712 1029811 'ToBeFree' 'ForkEvent' '' ('pull_merged', 0)]]
     repo_id actor_id    actor_login              type   action  \
0  159513310  9887585     snellingio        WatchEvent  started   
1  167922316  9887585     snellingio        WatchEvent  started   
2  170486632  9887585     snellingio        WatchEvent  started   
3  105722352  9887716       vpperego        WatchEvent  started   
4  158469400  9887743  ElliottMiller  PullRequestEvent   opened   

        pull_merged  
0  (pull_merged, 0)  
1  (pull_merged, 0)  
2  (pull_merged, 0)  
3  (pull_merged, 0)  
4  (pull_merged, 0)  


In [2]:
dset, repo_id = get_dataset(lambda x: np.log(1. + x))

tfidf_vec, sents = transform(dset.data, 100)

Now loading text data...
Now loading contribute vector data, this may take minutes.
Text data's length:  24287
Vector's shape:  (24287, 1638)
Complete 0 sentences.
Complete 1000 sentences.
Complete 2000 sentences.
Complete 3000 sentences.
Complete 4000 sentences.
Complete 5000 sentences.
Complete 6000 sentences.
Complete 7000 sentences.
Complete 8000 sentences.
Complete 9000 sentences.
Complete 10000 sentences.
Complete 11000 sentences.
Complete 12000 sentences.
Complete 13000 sentences.
Complete 14000 sentences.
Complete 15000 sentences.
Complete 16000 sentences.
Complete 17000 sentences.
Complete 18000 sentences.
Complete 19000 sentences.
Complete 20000 sentences.
Complete 21000 sentences.
Complete 22000 sentences.
Complete 23000 sentences.
Complete 24000 sentences.


In [3]:
w = 1.
print(tfidf_vec.shape, dset.vec.shape)
features = np.concatenate((tfidf_vec, w * dset.vec), axis = 1)
print(features.shape)
k = 20
HC = AgglomerativeClustering(n_clusters = k, affinity = 'cosine', linkage = 'average')
KM = KMeans(k, CosineDistance)
GM = GaussianMixture(n_components = k, max_iter = 1000)

KM.fit(features)
# HC.fit(features)
# GM.fit(features)
y = KM.labels
# y = GM.labels_
# y = HC.lables_
distribution = np.sum(np.ones((k, 1)) @ y.reshape(1, -1) == np.arange(k).reshape(k, 1) @ np.ones((1, y.shape[0])), axis = 1)
print(distribution)
print('Std of distribution: {std}'.format(std = np.std(distribution)))


(24287, 5000) (24287, 1638)
(24287, 6638)
[ 679  997 4115  421  903  446 1142 1316  649 1238  971  436  780  850
 3169  859  647 2694  986  989]
Std of distribution: 947.3021838357599


In [9]:
seq_vec = embedding_vec(sents[0: 10000], 10000)

In [4]:
silhouette_score(features,KM.labels,metric='cosine')

0.27363862575529285

In [8]:
torch.set_default_tensor_type(torch.FloatTensor)

class datasets():
    def __init__(self, X, y):
        self.x = X
        self.y = y
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return len(self.y)

class Model(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout = 0.5):
        # assert len(hidden_dims) == 3
        
        super().__init__()
        
        self.w1 = nn.Linear(input_dim, hidden_dims[0])
        self.w2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.w3 = nn.Linear(hidden_dims[1], hidden_dims[2])
        self.w4 = nn.Linear(hidden_dims[2], output_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
        nn.init.kaiming_normal_(self.w1.weight)
        nn.init.kaiming_normal_(self.w2.weight)
        nn.init.kaiming_normal_(self.w3.weight)
        nn.init.kaiming_normal_(self.w4.weight)
        
    def forward(self, X):

        l1 = self.relu(self.dropout(self.w1(X)))
        l2 = self.relu(self.dropout(self.w2(l1)))
        l3 = self.relu(self.dropout(self.w3(l2)))
        out = self.w4(l3)
        
        return out

class tcnn(nn.Module):
    def __init__(self, seq_len, vocab_size, output_dim, dropout = 0.5):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, 128)
        self.linear1 = nn.Linear(vocab_size, 128)
        self.conv = nn.Sequential(
            nn.Conv2d(1, 128, (2, 128)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((seq_len - 1, 1))
        )
        self.rnn = nn.LSTM(128, 256, 2, batch_first = True, bidirectional = True)
        self.linear2 = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(512, output_dim)
        ) 
        self.dropout = nn.Dropout(dropout)
        
        nn.init.kaiming_normal_(self.embedding.weight)
    
    def forward(self, x):
        
        if len(x.shape) < 3:
            x = self.embedding(x)
        # 64, 102, 10000
        else:
            x = self.linear1(x) 
        # 64, 102, 128
        # x = x.unsqueeze(1) # 64, 1, 102, 128
        # x = self.conv(x) # 64, 128, 2, 1
        x = self.rnn(x)[0][:, -1, :]
        # x = x.view(x.size(0), -1) # 64, 128
        x = self.dropout(x)
        x = self.linear2(x) # 64, 20
        return x

In [12]:
X_train = torch.tensor(features[0:24285], dtype = torch.float32)
y_train = torch.tensor(y[0:24285], dtype = torch.long)

# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1765)

num_folds = 5
NUM_TRAIN = 24285
num_train = NUM_TRAIN - NUM_TRAIN // num_folds
num_val = NUM_TRAIN - num_train

#loader_test = DataLoader(datasets(X_test, y_test), batch_size=64,
#                            sampler=sampler.SubsetRandomSampler(range(num_test)))

# print(y_train.shape)

X_train_folds = np.split(X_train, num_folds, axis = 0)
y_train_folds = np.split(y_train, num_folds, axis = 0)

input_dim = 6638
hidden_dims = [1024, 1024, 1024]
output_dim = 20


# 超参
lr = 2e-4 # 1e-2 for sgd
betas = (0.9, 0.999) # beta0 = 0.5，0.75, 0.99

# 最优模型
best_model = None
best_acc = 0

k = 0
X_train_fold = np.concatenate([ fold for j, fold in enumerate(X_train_folds) if k != j ])
y_train_fold = np.concatenate([ fold for j, fold in enumerate(y_train_folds) if k != j ])

loader_train = DataLoader(datasets(X_train_fold, y_train_fold), batch_size=64,
                      sampler=sampler.SubsetRandomSampler(range(num_train)))
loader_test = DataLoader(datasets(X_train_folds[k], y_train_folds[k]), batch_size=64,
                        sampler=sampler.SubsetRandomSampler(range(num_val)))

model = Model(input_dim, hidden_dims, output_dim, 0.5).cuda()
# model = tcnn(102, 10000, 20).cuda()

# 优化器
sgd = optim.SGD(model.parameters(), lr = lr, momentum = 0.9, weight_decay = 1e-5)
adam = optim.Adam(model.parameters(), lr = lr, betas = betas, weight_decay = 0.)

acc = train(model, adam, loader_train, epochs = 5)


print("\n\nResult on test dataset:")
check_accuracy(loader_test, model)

Iteration 0, loss = 2.99938
Got 2568 / 19428 correct (13.22) on validation dataset
Iteration 100, loss = 1.91899
Got 10232 / 19428 correct (52.67) on validation dataset
Iteration 200, loss = 0.62401
Got 18144 / 19428 correct (93.39) on validation dataset
Iteration 300, loss = 0.23786
Got 19032 / 19428 correct (97.96) on validation dataset
Iteration 0, loss = 0.31813
Got 19056 / 19428 correct (98.09) on validation dataset
Iteration 100, loss = 0.05084
Got 19156 / 19428 correct (98.60) on validation dataset
Iteration 200, loss = 0.12111
Got 19205 / 19428 correct (98.85) on validation dataset
Iteration 300, loss = 0.05347
Got 19336 / 19428 correct (99.53) on validation dataset
Iteration 0, loss = 0.05342
Got 19344 / 19428 correct (99.57) on validation dataset
Iteration 100, loss = 0.09797
Got 19362 / 19428 correct (99.66) on validation dataset
Iteration 200, loss = 0.07282
Got 19370 / 19428 correct (99.70) on validation dataset
Iteration 300, loss = 0.04832
Got 19401 / 19428 correct (99.8

In [8]:
# _, test_pred = model(X_train_folds[0]).max(1)
print(y[0: 20])

[18 12 10  0 16  0  5 14 10 16 16  2  0  8 13  6 10  0 11  5]


In [10]:
def train(model, optimizer, loader_train, loader_val = None, epochs = 1):
    acc = 0.
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            x, y = x.cuda(), y.cuda()
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()
            if t % 100 == 0:
                print('Iteration %d, loss = %.5f' % (t, loss.item()))
                if loader_val == None:
                    check_accuracy(loader_train, model)
                else:
                    acc = check_accuracy(loader_test, model, need_acc = True)
    if acc > 0:
        return acc
def check_accuracy(loader_val, model, loader_train = None, need_acc = False):
      
    num_correct_val = 0
    num_samples_val = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader_val:
            x, y = x.cuda(), y.cuda()
            scores = model(x)
            _, preds = scores.max(1)
            num_correct_val += (preds == y).sum()
            num_samples_val += preds.size(0)
        
        val_acc = float(num_correct_val) / num_samples_val
        print('Got %d / %d correct (%.2f) on validation dataset' % (num_correct_val, num_samples_val, 100 * val_acc))
        
    if need_acc:
        return 100 * val_acc

In [20]:
import matplotlib.pyplot as plt
import re
import jieba

from wordcloud import WordCloud

text3 = " ".join(np.array(fixed_data)[y == 3])

text7 = " ".join(np.array(fixed_data)[y == 7])

text9 = " ".join(np.array(fixed_data)[y == 9])

wc = WordCloud(

# 设置字体，不指定就会出现乱码

# 设置背景色

background_color='white',

# 设置背景宽

width=500,

# 设置背景高

height=350,

# 最大字体

max_font_size=50,

# 最小字体

min_font_size=10,

mode='RGBA'

#colormap='pink'

)

wc.generate(text9)
wc.to_file(r"wordcloud9.png")

In [1]:
import re
from nltk.corpus import stopwords
from transform import word_filter
d = 'asd555asd'
d = re.sub(r'[^a-zA-Z]', ' ', d)
s = 'A sh iter  aaaaaaaaaaaaaaaaaaaaaa adis distan is'
print(word_filter(s))

A [True, True, False, True, True]

sh [True, True, True, True, True]

iter [True, True, True, True, True]

aaaaaaaaaaaaaaaaaaaaaa [True, True, False, True, True]

adis [True, True, True, True, True]

distan [True, True, True, True, True]

is [True, True, True, True, False]

sh iter adis distan


In [9]:
print(np.array(repo_id)[[2,8,16]])

[ 388860  611562 1117948]


In [10]:
ids = (388860, 611562, 1117948)
sql = """
select distinct(repo_name) from ods_github_log
where pt > 20190101 and pt < 20200101
and repo_id in {list}
""".format(list = ids)
with o.execute_sql(sql).open_reader() as reader:
    for r in reader:
        print(r)

odps.Record {
  repo_name         'VsVim/VsVim'
}
odps.Record {
  repo_name         'holman/dotfiles'
}
odps.Record {
  repo_name         'jaredpar/VsVim'
}
odps.Record {
  repo_name         'kwgoodman/bottleneck'
}
odps.Record {
  repo_name         'pydata/bottleneck'
}
